## For TPC-DS dataset with the following query tempalte:
SELECT ss_store_sk, AF(ss_sales_price)

FROM store_sales

WHERE ss_sold_date_sk BETWEEN low AND high

GROUP BY ss_store_sk

In [1]:
from dbestclient.executor.executor import SqlExecutor

## Parameter Initialization

In [2]:
sqlExecutor = SqlExecutor()
sqlExecutor.execute("set csv_split_char='|'")
sqlExecutor.execute("set encoder='embedding'")
sqlExecutor.execute("set n_epoch=20")
sqlExecutor.execute(
    "set table_header="
    + "'ss_sold_date_sk|ss_sold_time_sk|ss_item_sk|ss_customer_sk|ss_cdemo_sk|ss_hdemo_sk|"
    + "ss_addr_sk|ss_store_sk|ss_promo_sk|ss_ticket_number|ss_quantity|ss_wholesale_cost|"
    + "ss_list_price|ss_sales_price|ss_ext_discount_amt|ss_ext_sales_price|"
    + "ss_ext_wholesale_cost|ss_ext_list_price|ss_ext_tax|ss_coupon_amt|ss_net_paid|"
    + "ss_net_paid_inc_tax|ss_net_profit|none'"
)

Local mode is on, as no slaves are provided.
start loading pre-existing models.
Loaded 6 models. time cost  0.306281 s
OK, csv_split_char is updated.
OK, encoder is updated.
OK, n_epoch is updated.
OK, table_header is updated.


## Model Creation

In [3]:
# sqlExecutor.execute("drop table template1")  # drop the model
sqlExecutor.execute(
    "create table template1(ss_sales_price real, ss_sold_date_sk real) from '/home/quincy/Documents/workspace/data/tpcds/1t/ss_1m.csv' GROUP BY ss_store_sk method uniform size 0.0003472"
)

Start creating model template1
The given table is treated as a uniform sample, and it is obtained with sampling rate 0.0003472
Reading data file...
get frequency info from data....
Starting training kde models for model template1
training regression...
********************************************************************************
finish training embedding.
embedding inference...
start normalizing data...
transform data from MDN training...
finish transforming data from MDN training...
< Epoch 0
< Epoch 1
< Epoch 2
< Epoch 3
< Epoch 4
< Epoch 5
< Epoch 6
< Epoch 7
< Epoch 8
< Epoch 9
< Epoch 10
< Epoch 11
< Epoch 12
< Epoch 13
< Epoch 14
< Epoch 15
< Epoch 16
< Epoch 17
< Epoch 18
< Epoch 19
Finish regression training.
training density...
********************************************************************************
finish training embedding.
< Epoch 0
< Epoch 1
< Epoch 2
< Epoch 3
< Epoch 4
< Epoch 5
< Epoch 6
< Epoch 7
< Epoch 8
< Epoch 9
< Epoch 10
< Epoch 11
< Epoch 12
< Epoch 1

## Query Serving

In [48]:
p_count = sqlExecutor.execute(
    "select ss_store_sk, count(ss_sales_price)  from template1 where   2451119  <=ss_sold_date_sk<= 2451483  group by ss_store_sk"
)
p_sum = sqlExecutor.execute(
    "select ss_store_sk, sum(ss_sales_price)  from template1 where   2451119  <=ss_sold_date_sk<= 2451483  group by ss_store_sk"
)
p_avg = sqlExecutor.execute(
    "select ss_store_sk, avg(ss_sales_price)  from template1 where   2451119  <=ss_sold_date_sk<= 2451483  group by ss_store_sk"
)

# sqlExecutor.execute("drop table template1")

OK
   0            1
     7.043875e+06
   1 1.136620e+06
  10 1.132917e+06
 100 1.170520e+06
1000 1.139759e+06
 103 1.112759e+06
 104 1.183720e+06
 106 1.124913e+06
 109 1.143446e+06
 110 1.160936e+06
 112 1.150763e+06
 115 1.178556e+06
 116 1.140365e+06
 118 1.146766e+06
 121 1.146736e+06
 122 1.187618e+06
 124 1.153004e+06
 127 1.184828e+06
 128 1.203284e+06
  13 1.127203e+06
 130 1.148989e+06
 133 1.155995e+06
 134 1.178507e+06
 136 1.128249e+06
 139 1.135676e+06
  14 1.183693e+06
 140 1.156919e+06
 142 1.192307e+06
 145 1.182470e+06
 146 1.210602e+06
 148 1.180108e+06
 151 1.115199e+06
 152 1.158713e+06
 154 1.153203e+06
 157 1.152852e+06
 158 1.170009e+06
  16 1.180790e+06
 160 1.203119e+06
 163 1.153530e+06
 164 1.083016e+06
 166 1.159244e+06
 169 1.112457e+06
 170 1.181515e+06
 172 1.172658e+06
 175 1.169676e+06
 176 1.152531e+06
 178 1.188765e+06
 181 1.107693e+06
 182 1.140557e+06
 184 1.169213e+06
 187 1.215547e+06
 188 1.132578e+06
  19 1.169057e+06
 190 1.108371e+06
 193 1.

## HIVE query
select ss_store_sk, count(ss_sales_price),sum(ss_sales_price),avg(ss_sales_price)  from store_sales_1t where  ss_sold_date_sk BETWEEN 2451119  AND 2451483  group by ss_store_sk

## Plot Distribution

In [88]:
import matplotlib
import matplotlib._color_data as mcd
import matplotlib.patches as mpatch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from matplotlib.ticker import FuncFormatter
import pandas as pd

%matplotlib widget


truth = pd.read_csv("t1.csv")


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')


# ax.scatter(truth["key"],[0]*len(truth),truth["count"],c='r',marker='^', label='COUNT From HIVE',alpha=0.5)
# ax.scatter(truth["key"],[1]*len(truth),truth["sum"],c='b',marker='o', label='SUM From HIVE',alpha=0.5)
ax.scatter(truth["key"],[2]*len(truth),truth["avg"],c='g',marker='*', label='AVG From HIVE',alpha=0.5)


p_count[0] = pd.to_numeric(p_count[0],errors='coerce')
p_sum[0] = pd.to_numeric(p_sum[0],errors='coerce')
p_avg[0] = pd.to_numeric(p_avg[0],errors='coerce')
# ax.scatter(p_count[0],[0]*len(p_count),p_count[1],c='c',marker='s', label='COUNT From DBEst++')
# ax.scatter(p_sum[0],[1]*len(p_sum),p_sum[1],c='m',marker='x', label='SUM From DBEst++')
ax.scatter(p_avg[0],[2]*len(p_avg),p_avg[1],c='y',marker='p', label='AVG From DBEst++')

ax.set_xlabel("Group Key")
ax.set_ylabel("Aggregate Type")
ax.set_zlabel("Query Result")
ax.set_yticks(np.arange(0, 3))
ax.set_yticklabels(["COUNT", "SUM","AVG"])
ax.set_title("Comparison of DBEst++ and HIVE")
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …